In [1]:
from azureml.core import Workspace, Experiment, Run
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal, BayesianParameterSampling, uniform, choice

## Test File Locally
We've created a script file called `train.py` that will train our ML model for us. It accepts a few different paramters - such as we'll see by running `python train.py --help`

We'll run the script locally to test if the script runs successfully. _Note: By default this will run in 'offline' mode - and nothing will actually be logged to AML. Instead, we'll see printed statements about `Attempted to log...`

In [2]:
%conda run -n py36 python train.py --help

usage: train.py [-h] --data-path DATA_PATH [--alpha ALPHA]
                [--file-name FILE_NAME] [--l1-ratio L1_RATIO]
                [--learning-rate LEARNING_RATE] [--model-name MODEL_NAME]

optional arguments:
  -h, --help            show this help message and exit
  --data-path DATA_PATH, -d DATA_PATH
                        The path where the data file is located
  --alpha ALPHA, -a ALPHA
                        The alpha value for training
  --file-name FILE_NAME, -f FILE_NAME
                        The file name of the diabetes csv dataset
  --l1-ratio L1_RATIO   The l1_ratio of the Scikit-Learn ElasticNet model
  --learning-rate LEARNING_RATE, -l LEARNING_RATE
                        The learning rate for the GBT model
  --model-name MODEL_NAME, -n MODEL_NAME
                        The name of the model to try. Supported calues are
                        'ridge', 'elastic', and 'gbt'

Note: you may need to restart the kernel to use updated packages.


In [3]:
%conda run -n py36 python train.py --data-path ../../data

Data contains 353 training samples and 89 test samples
Attempted to log scalar metric alpha:
0.03
Attempted to log scalar metric model_type:
Ridge
100%|███████████████████████████████████████████| 89/89 [00:06<00:00, 14.87it/s]
ExplanationClient.upload_model_explanation called.
Attempted to log scalar metric rmse:
58.52264788862529
Attempted to log scalar metric mae:
46.08348217820383
Attempted to log scalar metric r2:
0.33210821912058
Attempted to log image metric residuals-v-actuals:
Figure(640x480)
Attempted to log image metric residuals-histogram:
Figure(640x480)
Attempted to log image metric prediction-v-actual:
Figure(640x480)

Note: you may need to restart the kernel to use updated packages.


## Submit to AML Compute for Computation

In [4]:
experiment_name = 'diabetes_regression'
ws = Workspace.from_config()

# Attach to experiment
experiment = Experiment(workspace=ws, name=experiment_name)

In [5]:
# Define the datastore being used and the cpu_cluster being used
datastore = ws.datastores['diabetes']
cpu_cluster = ws.compute_targets['cpu-cluster']

In [6]:
args = {"--data-path": datastore.as_mount()}

est = Estimator(source_directory=".",
                compute_target=cpu_cluster,
                entry_script="train.py",
                script_params=args,
                conda_packages=['scikit-learn', 'seaborn', 'pandas'],
                pip_packages=['azureml-defaults', 'azureml-contrib-explain-model', 'azureml-explain-model']
                )

In [7]:
run = experiment.submit(est)

In [8]:
# the first time you run this, it might take a bit of time to build the container (took me 8 minutes in test)

from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Train Hyperparameters

In [11]:
sampled_params = BayesianParameterSampling({
                        '--alpha': uniform(0, 1),
                        '--l1-ratio': uniform(0, 1),
                        '--model-name': choice('ridge', 'elastic', 'gbt'),
                        '--learning-rate': uniform(0, 1)
                        })

hyper_run_config = HyperDriveConfig(estimator=est,
                                      hyperparameter_sampling=sampled_params,
                                      primary_metric_name='rmse',
                                      primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                      max_total_runs=50,
                                      max_concurrent_runs=5)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Current value for max_total_runs:50. Recommendend value:80.


In [ ]:
hd_run = experiment.submit(hyper_run_config)

In [13]:
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…